# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 16 2023 12:41PM,264900,10,Enova-11976,Emerginnova,Released
1,Jun 16 2023 12:40PM,264899,19,MSP222606,"Methapharm, Inc.",Released
2,Jun 16 2023 12:40PM,264899,19,MSP222759,"Methapharm, Inc.",Released
3,Jun 16 2023 12:40PM,264899,19,MSP222760,"Methapharm, Inc.",Released
4,Jun 16 2023 10:38AM,264868,15,9576810,"Woodfield Pharmaceutical, LLC",Executing
5,Jun 16 2023 10:23AM,264886,19,ADV80021294,"AdvaGen Pharma, Ltd",Released
6,Jun 16 2023 10:23AM,264886,19,ADV80021300,"AdvaGen Pharma, Ltd",Released
7,Jun 16 2023 10:23AM,264886,19,ADV80021301,"AdvaGen Pharma, Ltd",Released
8,Jun 16 2023 10:23AM,264886,19,ADV80021302,"AdvaGen Pharma, Ltd",Released
9,Jun 16 2023 10:23AM,264886,19,ADV80021303,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
6,264884,Executing,1
7,264886,Executing,22
8,264886,Released,16
9,264899,Released,3
10,264900,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264868,1.0,NaN
264884,1.0,NaN
264886,22.0,16.0
264899,NaN,3.0
264900,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264844,0.0,1.0
264845,0.0,1.0
264850,0.0,1.0
264857,1.0,0.0
264866,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264844,0,1
264845,0,1
264850,0,1
264857,1,0
264866,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264844,0,1
1,264845,0,1
2,264850,0,1
3,264857,1,0
4,264866,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264844,,1
1,264845,,1
2,264850,,1
3,264857,1,
4,264866,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 16 2023 12:41PM,264900,10,Emerginnova
1,Jun 16 2023 12:40PM,264899,19,"Methapharm, Inc."
4,Jun 16 2023 10:38AM,264868,15,"Woodfield Pharmaceutical, LLC"
5,Jun 16 2023 10:23AM,264886,19,"AdvaGen Pharma, Ltd"
43,Jun 16 2023 10:15AM,264884,19,"AdvaGen Pharma, Ltd"
44,Jun 15 2023 4:43PM,264866,12,"SD Head USA, LLC"
45,Jun 15 2023 3:54PM,264857,10,"NBTY Global, Inc."
46,Jun 15 2023 3:01PM,264850,20,Else Nutrition
47,Jun 15 2023 2:36PM,264845,10,"Senseonics, Incorporated"
48,Jun 15 2023 2:30PM,264844,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 16 2023 12:41PM,264900,10,Emerginnova,,1
1,Jun 16 2023 12:40PM,264899,19,"Methapharm, Inc.",,3
2,Jun 16 2023 10:38AM,264868,15,"Woodfield Pharmaceutical, LLC",1,
3,Jun 16 2023 10:23AM,264886,19,"AdvaGen Pharma, Ltd",22,16
4,Jun 16 2023 10:15AM,264884,19,"AdvaGen Pharma, Ltd",1,
5,Jun 15 2023 4:43PM,264866,12,"SD Head USA, LLC",1,
6,Jun 15 2023 3:54PM,264857,10,"NBTY Global, Inc.",1,
7,Jun 15 2023 3:01PM,264850,20,Else Nutrition,,1
8,Jun 15 2023 2:36PM,264845,10,"Senseonics, Incorporated",,1
9,Jun 15 2023 2:30PM,264844,10,"Senseonics, Incorporated",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 16 2023 12:41PM,264900,10,Emerginnova,1,
1,Jun 16 2023 12:40PM,264899,19,"Methapharm, Inc.",3,
2,Jun 16 2023 10:38AM,264868,15,"Woodfield Pharmaceutical, LLC",,1
3,Jun 16 2023 10:23AM,264886,19,"AdvaGen Pharma, Ltd",16,22
4,Jun 16 2023 10:15AM,264884,19,"AdvaGen Pharma, Ltd",,1
5,Jun 15 2023 4:43PM,264866,12,"SD Head USA, LLC",,1
6,Jun 15 2023 3:54PM,264857,10,"NBTY Global, Inc.",,1
7,Jun 15 2023 3:01PM,264850,20,Else Nutrition,1,
8,Jun 15 2023 2:36PM,264845,10,"Senseonics, Incorporated",1,
9,Jun 15 2023 2:30PM,264844,10,"Senseonics, Incorporated",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 16 2023 12:41PM,264900,10,Emerginnova,1,
1,Jun 16 2023 12:40PM,264899,19,"Methapharm, Inc.",3,
2,Jun 16 2023 10:38AM,264868,15,"Woodfield Pharmaceutical, LLC",,1
3,Jun 16 2023 10:23AM,264886,19,"AdvaGen Pharma, Ltd",16,22
4,Jun 16 2023 10:15AM,264884,19,"AdvaGen Pharma, Ltd",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 16 2023 12:41PM,264900,10,Emerginnova,1.0,NaN
1,Jun 16 2023 12:40PM,264899,19,"Methapharm, Inc.",3.0,NaN
2,Jun 16 2023 10:38AM,264868,15,"Woodfield Pharmaceutical, LLC",NaN,1.0
3,Jun 16 2023 10:23AM,264886,19,"AdvaGen Pharma, Ltd",16.0,22.0
4,Jun 16 2023 10:15AM,264884,19,"AdvaGen Pharma, Ltd",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 16 2023 12:41PM,264900,10,Emerginnova,1.0,0.0
1,Jun 16 2023 12:40PM,264899,19,"Methapharm, Inc.",3.0,0.0
2,Jun 16 2023 10:38AM,264868,15,"Woodfield Pharmaceutical, LLC",0.0,1.0
3,Jun 16 2023 10:23AM,264886,19,"AdvaGen Pharma, Ltd",16.0,22.0
4,Jun 16 2023 10:15AM,264884,19,"AdvaGen Pharma, Ltd",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1059446,3.0,1.0
12,264866,0.0,1.0
15,264868,0.0,1.0
19,794669,19.0,23.0
20,264850,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1059446,3.0,1.0
1,12,264866,0.0,1.0
2,15,264868,0.0,1.0
3,19,794669,19.0,23.0
4,20,264850,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,3.0,1.0
1,12,0.0,1.0
2,15,0.0,1.0
3,19,19.0,23.0
4,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,3.0
1,12,Released,0.0
2,15,Released,0.0
3,19,Released,19.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20
Status,,,,,
Executing,1.0,1.0,1.0,23.0,0.0
Released,3.0,0.0,0.0,19.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20
0,Executing,1.0,1.0,1.0,23.0,0.0
1,Released,3.0,0.0,0.0,19.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20
0,Executing,1.0,1.0,1.0,23.0,0.0
1,Released,3.0,0.0,0.0,19.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()